**SOW-MKI49: Neural Information Processing Systems**  
*Weeks 4 and 5: Assignment (225 points + 30 bonus points)*  
Author: Umut

In [0]:
# Group number: 17
# Sameera Sandaruwan, s1014012
# Student 2 name, student 2 number: ...
# Student 3 name, student 3 number: ...

In [0]:
from chainer import ChainList, optimizers, serializers
import chainer
import chainer.functions as F
import chainer.links as L
import numpy as np
import cupy

**WaveNet component (75 points)**

* Implement missing parts of the call method (y and z). **25 points**
* Implement residual block class. **50 points**

---
Reminder:

* One convolution layer that has 61 kernels of size 2 with no nonlinearities.
![alt text](http://i67.tinypic.com/21mgi2w.png)
![alt text](http://i67.tinypic.com/292n04y.png)
---



In [0]:
class _ResidualBlock(ChainList):
    def __init__(self, dilation):
        super(_ResidualBlock, self).__init__(
            L.Convolution2D(in_channels=61, out_channels=2*61, ksize=(1, 2), dilate=dilation),
            L.Convolution2D(in_channels=61, out_channels=512, ksize=(1, 1)),
            L.Convolution2D(in_channels=61, out_channels=61, ksize=(1, 1))
        )
        
    
    def __call__(self, x):
        h = F.split_axis(self[0](x), 2, 1)
        gatedOutput = F.sigmoid(h[0]) * F.tanh(h[1])
        z = self[1](gatedOutput)
        y = self[2](gatedOutput) + x
        return y, z

    
class _WaveNet(ChainList):
    def __init__(self):
        links = (L.Convolution2D(in_channels=61, out_channels=61, ksize=(1, 2)),)
        links += tuple(_ResidualBlock((1, 2 ** (i % 6))) for i in range(6))
        links += (L.Convolution2D(512, 512, 1), L.Convolution2D(512, 3843, 1))
        super(_WaveNet, self).__init__(*links)
        '''
        QUESTIONS
        1. Why/how conv2D_7 has input of 512? If 512, should the last Res. layer has output of 512
        2. Why/How 61*61 and 2*61 comes about?
        '''
        ''' --- Structure --- (in > out)
        0.conv2D (61 -> 61)
        1.Residual (61 > 61, 512)
        2.Residual (61 > 61, 512)
        3.Residual (61 > 61, 512)
        4.Residual (61 > 61, 512)
        5.Residual (61 > 61, 512)
        6.Residual (61 > 61, 512)
        7.conv2D (512 > 512)
        8.conv2D (512 > 3843)
        '''
        
    def __call__(self, x):
        y = self[0](F.pad(x, ((0, 0), (0, 0), (0, 0), (1, 0)), 'constant'))
        z = 0
        # z - skip connection output
        # y - output of each res. layer which becomes the input to next res. layer
        
        for i in range(1, len(self) - 2):#going through Residual layers
            #pass
            y, z_hat = self[i](y)
            z += z_hat
        
        z = F.relu(self[7](z))
        z = self[8](z)
        y, z = F.split_axis(z, [61*61], 1) #if second param is an array, those positions are used as split points

        return F.reshape(y, (y.shape[0], 61, 61, y.shape[3])), F.reshape(z, (z.shape[0], 2, 61, z.shape[3]))
        # check shape sizes

**CRF-RNN component (50 points)**

* Implement missing parts of the call method (z). **25 points**
* Why is z not normalized in the last iteration? **25 points**

---

Reminder:

![alt text](http://i68.tinypic.com/sy6mix.png)

---

In [0]:
'''
CRF - Conditional random field
This class calculate Mean-field approximation
'''
class _CRF(ChainList):
    def __init__(self):
        super(_CRF, self).__init__(L.ConvolutionND(1, 2, 2, 1, nobias = True))

    def __call__(self, x, y):
        z = F.softmax(-y)

        for i in range(5):
            z = -y - self[0](F.batch_matmul(z, x))

            if i < 4:
                z = F.softmax(z)

        return z

**WaveCRF model (50 points)**

1. Implement missing parts of the call method (k, psi_u and Q_hat). **20 points**
2. Implement missing parts of the save and load methods (save and load model). **10 points**
3. Implement missing parts of the test and train methods (forward and/or backward propagate). **20 points**

In [0]:
class WaveCRF(object):
    def __init__(self):
        self.log = {('test', 'accuracy'): (), ('test', 'loss'): (), ('training', 'accuracy'): (),
                    ('training', 'loss'): ()}
        self.model = ChainList(_WaveNet(), _CRF())
        self.optimizer = optimizers.Adam(0.0002, 0.5)

        self.optimizer.setup(self.model)

    def __call__(self, x):
        k, psi_u = self.model[0](x)
        Q_hat = self.model[1](F.reshape(F.transpose(k, (0, 3, 1, 2)), (-1, 61, 61)),
                              F.reshape(F.transpose(psi_u, (0, 3, 1, 2)), (-1, 2, 61)))

        return F.transpose(F.reshape(Q_hat, (x.shape[0], x.shape[3], 2, 61)), (0, 2, 3, 1))

    @classmethod
    def load(cls, directory):
        self = cls()
        self.log = np.load('{}/log.npy'.format(directory))

        serializers.load_npz('{}/model.npz'.format(directory), self.model)
        serializers.load_npz('{}/optimizer.npz'.format(directory), self.optimizer)

        return self

    def save(self, directory):
        np.save('{}/log.npy'.format(directory), self.log)
        serializers.save_npz('{}/model.npz'.format(directory), self.model)
        serializers.save_npz('{}/optimizer.npz'.format(directory), self.optimizer)

    def test(self, Q, x):
        with chainer.using_config('train', False):
            Q_hat = self(x)
            loss = F.softmax_cross_entropy(Q_hat, Q)

            self.log['test', 'accuracy'] += (float(F.accuracy(Q_hat, Q).data),)
            self.log['test', 'loss'] += (float(loss.data),)

    def train(self, Q, x):
        Q_hat = self(x)
        loss = F.softmax_cross_entropy(Q_hat, Q)

        self.model.cleargrads()
        loss.backward()
        self.optimizer.update()

        self.log['training', 'accuracy'] += (float(F.accuracy(Q_hat, Q).data),)
        self.log['training', 'loss'] += (float(loss.data),)

# Training

In [0]:
%matplotlib inline

import IPython
import chainer
import matplotlib
import numpy
import os
import pickle
import random
import tqdm
import matplotlib.pyplot as plt

In [0]:
batch_size = 30
epochs = 100
root = 'WaveNetModel'
song_id = 4 # change for each new test

In [0]:
with open('Data/piano_rolls.p', 'rb') as f:
    piano_rolls = pickle.load(f, encoding = 'latin1') #works

keys = sorted(piano_rolls.keys())

random.seed(6)
random.shuffle(keys)

test_set = dict((key, piano_rolls[key]) for key in keys[:int(0.1 * len(keys))])
training_set = dict((key, piano_rolls[key]) for key in keys[int(0.1 * len(keys)):])
training_set_keys = list(training_set.keys())

In [0]:
waveCRF = WaveCRF()

waveCRF.model.to_gpu()

In [0]:
for epoch in tqdm.tnrange(epochs, ascii=True, desc="NIPS-G17:"):
    random.shuffle(training_set_keys)

    batch = ()

    for key in tqdm.tqdm_notebook(training_set_keys, leave = False, desc='Train {}:'.format(epoch)):
        i = random.randint(0, training_set[key].shape[1] - 80)
        batch += (training_set[key][32 : 93, i : i + 80],)

        if len(batch) == batch_size:
            batch = waveCRF.model.xp.array(batch)

            waveCRF.train(batch[:, :, 1:].astype('i'), batch[:, :, None, :-1].astype('f'))

            batch = ()

    for key in tqdm.tqdm_notebook(test_set, leave = False, desc='Val {}:'.format(epoch)):
        batch = waveCRF.model.xp.array((test_set[key][32 : 93],))

        waveCRF.test(batch[:, :, 1:].astype('i'), batch[:, :, None, :-1].astype('f'))

    IPython.display.clear_output()

    for i, key in enumerate(waveCRF.log):
        matplotlib.pyplot.subplot(221 + i)
        matplotlib.pyplot.plot(numpy.array(waveCRF.log[key]).reshape(epoch + 1, -1).mean(1))
        matplotlib.pyplot.xlabel('iteration')
        matplotlib.pyplot.ylabel(key)

    matplotlib.pyplot.tight_layout()
    matplotlib.pyplot.show()
    os.makedirs('{}/Models/WaveCRF_{}/{}'.format(root, song_id, epoch))
    waveCRF.save('{}/Models/WaveCRF_{}/{}'.format(root, song_id, epoch))

print("DONE...")

**Test (50 points)**  

* Generate a number of samples, pick the best one and play it in the notebook. **50 points**

# Test

***One Time***

In [ ]:
# Load model
model_dir = 'WaveNetModel/Models/WaveCRF_{}/{}'.format(song_id, epochs-1)
print(model_dir)
waveCRF_test = WaveCRF()
waveCRF_test.load(model_dir)
waveCRF_test.model.to_gpu()

#create a sample from the test set
batch = ()

for key in tqdm.tqdm_notebook(test_set_keys[:1], leave = False):
    i = random.randint(0, test_set[key].shape[1] - 80)
    Tsample = test_set[key][32 : 93, i : i + 80]
    batch += (Tsample,)

        
# print("---")
# print(batch)
batch = waveCRF_test.model.xp.array(batch)
# print(type(batch))
# print(batch.shape)
Xt = batch[:, :, None, :-1].astype('f')
Xt = F.pad(Xt, ((0, 0), (0, 0), (0, 0), (1, 0)), 'constant')
# print('1',Xt.shape)
# print('2',type(Xt))

# plot input song
input_song = cupy.asnumpy(batch[0])

# print(input_song.shape)
# print(type(input_song))
# print(input_song)

fig=plt.figure(figsize=(10, 10))
plt.subplot(121)
plt.imshow(input_song)
plt.xlabel('Time steps (samples)')
plt.ylabel("Notes")

results_dir = 'Results'

np.savez('{}/input_song_{}.npz'.format(results_dir, song_id), input_song=input_song)


# prediction
prediction = F.softmax(waveCRF_test(Xt))

# decode prediction
new_note = prediction.data[:,:,:,-1] #slide out the predicted note - cupy.asnumpy()

# argmax
pred_notes = np.argmax(new_note, axis=1)>0

# add predicted note to the previous note
Yt = F.pad(Xt, ((0, 0), (0, 0), (0, 0), (0, 1)), 'constant')
Yt.data[:,:,:,-1] = pred_notes[0].astype('f').reshape(1, 61, 1)

# print('3',Yt.data.shape)

# prep next Xt
next_Xt = Yt.data>0
# print('4',type(next_Xt))
next_Xt_crop = cupy.asnumpy(next_Xt[0].transpose([1,0,2])[0,:,-80:])
# print('5',next_Xt_crop.shape)
batch = (next_Xt_crop,)

batch = waveCRF_test.model.xp.array(batch)
Xt = batch[:, :, None, :-1].astype('f')
Xt = F.pad(Xt, ((0, 0), (0, 0), (0, 0), (1, 0)), 'constant')
# print('6',Xt.shape)

newImg = cupy.asnumpy(Yt.data)
newImg = newImg.transpose([0, 2, 1, 3])
plt.subplot(122)
plt.imshow(newImg[0][0])

***New Note generation loop***

In [ ]:
num_new_notes = 20
for _ in range(num_new_notes):
    # prediction
    prediction = F.softmax(waveCRF_test(Xt))

    # decode prediction
    new_note = prediction.data[:,:,:,-1] #slide out the predicted note - cupy.asnumpy()
    # print(new_note)
    # argmax
    pred_notes = np.argmax(new_note, axis=1)>0
    # print(pred_notes)
    # add predicted note to the previous note
    Yt = F.pad(Yt, ((0, 0), (0, 0), (0, 0), (0, 1)), 'constant')
    Yt.data[:,:,:,-1] = pred_notes[0].astype('f').reshape(1, 61, 1)

#     print(Yt.data.shape)

    # prep next Xt
    next_Xt = Yt.data>0
#     print(type(next_Xt))
    next_Xt_crop = cupy.asnumpy(next_Xt[0].transpose([1,0,2])[0,:,-80:])
#     print(next_Xt_crop.shape)
    batch = (next_Xt_crop,)

    batch = waveCRF_test.model.xp.array(batch)
    Xt = batch[:, :, None, :-1].astype('f')
    Xt = F.pad(Xt, ((0, 0), (0, 0), (0, 0), (1, 0)), 'constant')

print(Yt.data.shape)
newImg = cupy.asnumpy(Yt.data)
newImg = newImg.transpose([0, 2, 1, 3])[0][0]>0
fig=plt.figure(figsize=(10, 10))
plt.imshow(newImg)

**Bonus question (30 points)**

* Discuss how you can improve the model (you can talk about different architectures or different ways to encode the inputs, etc.) **10 points**
* Discuss the assumptions behind the meanfield approximation and its shortcomings. **10 points**
* Prove that the iterative update equation (CRF-RNN component) is differentiable so that we can backpropagate through them. **10 points**

1. swtch to my structure
2. output generation ??
3. output normalize & thresholding
4. output format